<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [ ]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC, Comp, Range, todf, report, tuple_keys
from tools.trust_ncg import minimize
import tools.che as che

In [ ]:
R=8.314 # J/(mol K)
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c):
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    W_n_vap = Pw * V_vap / (R * c.T)
    n_vap = jnp.concatenate((c.air_n, W_n_vap))
    P = jnp.sum(n_vap) * R * c.T / V_vap
    return (s.P_f - P)**2

s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.air_y * s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = 360


vsc=VSC(v,s)
model_f = vsc.transform(model)
model_f(vsc.x)

In [ ]:
modelf = jax.jit(vsc.transform(model))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

In [ ]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()


In [ ]:
def model2(c):
    r=DotMap()
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    r.W_n_vap = Pw * V_vap / (R * c.T)
    r.n_vap = jnp.concatenate((c.air_n, r.W_n_vap))
    P = jnp.sum(r.n_vap) * R * c.T / V_vap
    return ((s.P_f - P)/s.P_f)**2 + (r.W_n_vap - 0.3)**2, r

In [ ]:
s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.air_y * s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, 300, 400)
v.V_tot = Range(0.015, 0., 0.03)

vsc=VSC(v,s)

In [ ]:
modelf = jax.jit(vsc.transform(model2))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

In [ ]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()


In [ ]:
report(model2, vsc.xtoc(x_min))

In [ ]:
df=todf(s)

vector3     vector1
          1 2 3       1
P_i              100000
T_i                 298